In [29]:
import csv
import mysql.connector
import json
import logging

In [30]:
class MySQLLoader:
    
    def __init__(self, config_file_path):
        #reding the config file as a dict
        self.config = self.read_config(config_file_path)
        #connect to mysql server
        if self.config:
            self.mysql_connection = mysql.connector.connect(
                host=self.config['mysql_host'],
                user=self.config['mysql_user'],
                password=self.config['mysql_password'],
                database=self.config['mysql_database']
            )
            self.cursor = self.mysql_connection.cursor()

            # Configure logger
            logging.basicConfig(filename='mysql_loader.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    def __del__(self):
        #making sure to close the connection even if the user doesn't do it
        self.close_connection()
    
   
    def get_mysql_connection(self):
        return self.mysql_connection
    
    def get_cursor(self):
        return self.cursor
    
    '''
    user can send in a query and it will be exectued and ran
    '''
    def execute_query(self, query):
        try:
            self.cursor.execute(query)
            self.mysql_connection.commit()
        except Exception as e:
            logging.error(f"Error executing given query : {e}")
    
    #class to close connection to the mysql server
    def close_connection(self):
        if self.mysql_connection.is_connected():
            self.cursor.close()
            self.mysql_connection.close()
            logging.info("MySQL connection closed.")
            
    #the given config file is opened here
    def read_config(self, file_path):
        try:
            with open(file_path, 'r') as config_file:
                config = json.load(config_file)
            return config
        except Exception as e:
            logging.error(f"Error reading configuration: {e}")
            return None
        
    '''
    the majority of the work is done here.
    we first open the the csv file given a path.
    using the table name given is what will be CREATED in the server.
    assuming the first line in the csv file is the header that will be the column name
    continue reading the csv file and load each row into the table 
    log each step and any exeption that is thrown
    '''
    def load_csv_to_mysql(self, csv_file_path, table_name):
        try:
            # Read CSV file
            with open(csv_file_path, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                header = next(csv_reader)  # Assume the first row is header
                
                # writing a query to check 
                create_table_query = f"CREATE TABLE {table_name} ({', '.join(f'{col} VARCHAR(255)' for col in header)});"
                self.cursor.execute(create_table_query)
                logging.info(f"Table '{table_name}' created or already exists.")
           
                # Load data into MySQL
                for row in csv_reader:
                    columns = ', '.join(header)
                    values = ', '.join(f"'{val}'" for val in row)
                    query = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

                    self.cursor.execute(query)
                logging.info(f"Data from '{csv_file_path}' loaded into '{table_name}'.")
                
                # Commit the changes
                self.mysql_connection.commit()
                logging.info(f"Data from '{csv_file_path}' committed into '{table_name}'.")
                
        except Exception as e:
            logging.error(f"Error loading CSV to MySQL: {e}")


In [31]:
def main():
    
    config_file_path = 'config.json'
    loader = MySQLLoader(config_file_path)

    if loader.config:
        loader.load_csv_to_mysql(loader.config['product_csv_path'], 'Products')
        loader.load_csv_to_mysql(loader.config['sales_csv_path'], 'Sales')

    # making the view
        
    query = """
        CREATE OR REPLACE VIEW ProductsSalesView AS
        SELECT Sales.Day, Products.Brand, Sales.Sales
        FROM Sales
        INNER JOIN Products ON Sales.Product = Products.Product;
        """
    loader.execute_query(query)
    
    #getting a cursor to fetch the view and download it as a csv file
    mycursor = loader.get_cursor()
    mycursor.execute("SELECT * FROM ProductsSalesView")

    # Fetch data from the cursor
    data = mycursor.fetchall()
    
    #getting the header 
    header = [desc[0] for desc in mycursor.description]
    
    with open('ProductSalesView.csv', 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            # Write header
            csv_writer.writerow(header)
            
            # Write data rows
            csv_writer.writerows(data)

In [32]:
if __name__ == '__main__':
    main()

doing product
doing sales


In [33]:
import unittest
import os
import json
import csv
from unittest.mock import patch, MagicMock
import mysql.connector  # Make sure to have this library installed
#from your_module import MySQLLoader  # Replace 'your_module' with the actual module name

class TestMySQLLoader(unittest.TestCase):
    def setUp(self):
        # Create a temporary config file for testing
        self.config_file_path = 'test_config.json'
        with open(self.config_file_path, 'w') as f:
            f.write('{"mysql_host": "localhost", "mysql_user": "test_user", "mysql_password": "test_password", "mysql_database": "test_database"}')

    def tearDown(self):
        # Delete the temporary config file after testing
        os.remove(self.config_file_path)
    
    @patch()
    def test_init(self, mock_connect):
        # Mock the connection to MySQL
        mock_connection = mock_connect.return_value
        mock_cursor = mock_connection.cursor.return_value

        # Create an instance of MySQLLoader
        mysql_loader = MySQLLoader(self.config_file_path)

        # Assertions
        self.assertEqual(mysql_loader.mysql_connection, mock_connection)
        self.assertEqual(mysql_loader.cursor, mock_cursor)

        # Ensure that read_config is called during initialization
        mock_read_config = patch.object(MySQLLoader, 'read_config', autospec=True)
        mock_read_config.assert_called_once_with(self.config_file_path)

        # Ensure that logging is configured
        logging_config = logging.basicConfig.assert_called_once_with(filename='mysql_loader.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    def test_read_config(self, mock_json_load):
        # Mock the json.load function
        mock_config = {'mysql_host': 'localhost', 'mysql_user': 'test_user', 'mysql_password': 'test_password', 'mysql_database': 'test_database'}
        mock_json_load.return_value = mock_config

        # Create an instance of MySQLLoader
        mysql_loader = MySQLLoader(self.config_file_path)

        # Call the read_config method
        config = mysql_loader.read_config(self.config_file_path)

        # Assertions
        mock_json_load.assert_called_once_with(mock.ANY)
        self.assertEqual(config, mock_config)

    def test_load_csv_to_mysql(self, mock_commit, mock_execute, mock_csv_reader):
        # Mock the necessary methods
        mock_cursor = MagicMock()
        mock_connection = MagicMock()
        mock_cursor.return_value = mock_cursor
        mock_connection.return_value = mock_connection
        mock_csv_reader.return_value = [['value1', 'value2', 'value3']]

        # Create an instance of MySQLLoader
        mysql_loader = MySQLLoader(self.config_file_path)
        mysql_loader.cursor = mock_cursor
        mysql_loader.mysql_connection = mock_connection

        # Call the load_csv_to_mysql method
        csv_file_path = 'test.csv'
        table_name = 'test_table'
        mysql_loader.load_csv_to_mysql(csv_file_path, table_name)

        # Assertions
        mock_csv_reader.assert_called_once_with(mock.ANY)
        mock_execute.assert_called_once_with(mock.ANY)
        mock_commit.assert_called_once()

        
unittest.main()


E
ERROR: /Users/anderosaweeros/Library/Jupyter/runtime/kernel-e2371358-1631-467f-9ee6-b91ae853fd69 (unittest.loader._FailedTest./Users/anderosaweeros/Library/Jupyter/runtime/kernel-e2371358-1631-467f-9ee6-b91ae853fd69)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/anderosaweeros/Library/Jupyter/runtime/kernel-e2371358-1631-467f-9ee6-b91ae853fd69'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/Users/anderosaweeros/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
